In [ ]:
from PIL import Image
import os

def create_gif_from_frames(folder_path, output_gif, fps=10):
    """
    Crea un GIF a partir de una carpeta con frames (.png).
    
    Args:
        folder_path (str): Ruta de la carpeta con los frames.
        output_gif (str): Nombre del GIF de salida (ej. 'output.gif').
        fps (int): Cuántos frames por segundo (mayor valor = animación más rápida).
    """
    frames = sorted(
        [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(".png")]
    )

    if not frames:
        print("❌ No se encontraron archivos .png en la carpeta.")
        return

    images = [Image.open(frame) for frame in frames]
    
    # Guardar el GIF
    images[0].save(output_gif, save_all=True, append_images=images[1:], duration=1000//fps, loop=0)
    print(f"✅ GIF guardado como {output_gif}")

# 📌 Uso:
create_gif_from_frames(frames_path, "output.gif", fps=10)


✅ GIF guardado como output.gif


In [6]:
from PIL import Image
import os
import re

def sorted_nicely(file_list):
    """ Ordena archivos con números correctamente (ej. frame_1.png, frame_2.png, ...). """
    def convert(text):
        return int(text) if text.isdigit() else text.lower()
    
    def alphanum_key(key):
        return [convert(c) for c in re.split('([0-9]+)', key)]
    
    return sorted(file_list, key=alphanum_key)

def create_gif_from_frames(folder_path, output_gif, fps=10):
    """
    Crea un GIF a partir de una carpeta con frames (.png), asegurando el orden correcto.

    Args:
        folder_path (str): Ruta de la carpeta con los frames.
        output_gif (str): Nombre del GIF de salida (ej. 'output.gif').
        fps (int): Cuántos frames por segundo (mayor valor = animación más rápida).
    """
    frames = sorted_nicely(
        [f for f in os.listdir(folder_path) if f.endswith(".png")]
    )

    if not frames:
        print("❌ No se encontraron archivos .png en la carpeta.")
        return

    images = [Image.open(os.path.join(folder_path, frame)) for frame in frames]

    # Guardar el GIF
    images[0].save(output_gif, save_all=True, append_images=images[1:], duration=1000//fps, loop=0)
    print(f"✅ GIF guardado como {output_gif}")

# 📌 Uso:
frames_path = '/Users/arnaubarrera/Desktop/MSc Computer Vision/C5. Visual Recognition/mcv-c5-group-3/segmentation_results/segmentation_result'
create_gif_from_frames(frames_path, "output.gif", fps=10)


✅ GIF guardado como output.gif


In [ ]:
import os
import json
import glob
import numpy as np
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from typing import Dict, Optional, List, Union
from datetime import datetime

def calculate_global_map_from_files(
    predictions_dir: str,
    ground_truth_dir: str,
    output_file: str = "instance_segmentation_results.txt",
    matching_method: str = "index",
    video_ids: Optional[List[str]] = None
) -> Dict:
    """
    Calcula métricas globales de instance segmentation y genera un archivo TXT con los resultados.
    
    Args:
        predictions_dir: Directorio que contiene los archivos JSON de predicciones.
        ground_truth_dir: Directorio que contiene los archivos JSON de ground truth.
        output_file: Ruta del archivo de salida TXT (default: "instance_segmentation_results.txt").
        pred_pattern: Patrón glob para identificar archivos de predicciones (default: "coco_preds_*.json").
        gt_pattern: Patrón glob para identificar archivos de ground truth (default: "gt_coco_*.json").
        matching_method: Método para emparejar archivos ("index" o "name").
        video_ids: Lista opcional de IDs de vídeo para filtrar (e.j. ['0000', '0001', '0010']).
                   Si es None, se procesarán todos los vídeos.
    
    Returns:
        Dict: Diccionario con métricas globales.
    """
    pred_pattern = "preds_coco_*.json"
    gt_pattern = "gt_coco_*.json"
    
    # Obtener listas de archivos
    pred_files = sorted(glob.glob(os.path.join(predictions_dir, pred_pattern)))
    gt_files = sorted(glob.glob(os.path.join(ground_truth_dir, gt_pattern)))
    
    if not pred_files:
        raise ValueError(f"No se encontraron archivos de predicciones con el patrón {pred_pattern} en {predictions_dir}")
    if not gt_files:
        raise ValueError(f"No se encontraron archivos de ground truth con el patrón {gt_pattern} en {ground_truth_dir}")
    
    print(f"Encontrados {len(pred_files)} archivos de predicciones y {len(gt_files)} archivos de ground truth")
    
    # Emparejar archivos de predicciones con ground truth
    paired_files = []
    
    if matching_method == "index":
        # Extraer índices numéricos de los nombres de archivo
        pred_indices = {}
        gt_indices = {}
        
        for pred_file in pred_files:
            # Extraer el índice (ej: "coco_preds_0000.json" -> "0000")
            basename = os.path.basename(pred_file)
            index_part = basename.split("_")[-1].split(".")[0]
            pred_indices[index_part] = pred_file
        
        for gt_file in gt_files:
            # Extraer el índice (ej: "gt_coco_0000.json" -> "0000")
            basename = os.path.basename(gt_file)
            index_part = basename.split("_")[-1].split(".")[0]
            gt_indices[index_part] = gt_file
        
        # Determinar los índices a procesar
        if video_ids:
            # Filtrar por los IDs de vídeo proporcionados
            valid_indices = [idx for idx in video_ids if idx in pred_indices and idx in gt_indices]
            if not valid_indices:
                raise ValueError(f"Ninguno de los video_ids proporcionados ({video_ids}) corresponde a archivos válidos")
            print(f"Filtrando por {len(valid_indices)} IDs de vídeo especificados")
        else:
            # Usar todos los índices coincidentes
            valid_indices = set(pred_indices.keys()) & set(gt_indices.keys())
        
        # Emparejar por índice
        paired_files = [(pred_indices[idx], gt_indices[idx]) for idx in sorted(valid_indices)]
        
    elif matching_method == "name":
        # Emparejar por nombre base
        pred_dict = {os.path.splitext(os.path.basename(f))[0]: f for f in pred_files}
        gt_dict = {os.path.splitext(os.path.basename(f))[0]: f for f in gt_files}
        
        # Encontrar pares coincidentes, filtrando por video_ids si es necesario
        for pred_name, pred_file in pred_dict.items():
            for gt_name, gt_file in gt_dict.items():
                # Extraer ID del vídeo del nombre del archivo
                if "_" in pred_name:
                    video_id = pred_name.split("_")[-1]
                else:
                    video_id = pred_name
                
                if pred_name == gt_name.replace("gt_", ""):
                    # Si se proporcionaron video_ids, verificar si este vídeo está en la lista
                    if video_ids and video_id not in video_ids:
                        continue
                    
                    paired_files.append((pred_file, gt_file))
                    break
    else:
        raise ValueError(f"Método de emparejamiento no reconocido: {matching_method}")
    
    if not paired_files:
        if video_ids:
            raise ValueError(f"No se encontraron parejas coincidentes para los IDs de vídeo: {video_ids}")
        else:
            raise ValueError("No se encontraron parejas coincidentes de archivos de predicción y ground truth")
    
    print(f"Se emparejaron {len(paired_files)} archivos")
    
    # Guardar lista de archivos procesados
    processed_files = [f"- {os.path.basename(p)} ↔ {os.path.basename(g)}" for p, g in paired_files]
    
    # Inicializar estructuras para datos globales
    global_predictions = []
    global_gt = {
        'images': [],
        'annotations': [],
        'categories': []
    }
    
    next_image_id = 1
    next_annotation_id = 1
    image_id_mapping = {}  # Mapeo de IDs originales a IDs globales
    
    # Procesar cada pareja de archivos
    for pred_file, gt_file in paired_files:
        print(f"Procesando: {os.path.basename(pred_file)} y {os.path.basename(gt_file)}")
        
        # Cargar predicciones
        with open(pred_file, 'r') as f:
            video_predictions = json.load(f)
        
        # Cargar ground truth
        with open(gt_file, 'r') as f:
            gt_data = json.load(f)
        
        # Extraer categorías del primer GT si aún no las tenemos
        if not global_gt['categories'] and 'categories' in gt_data:
            global_gt['categories'] = gt_data['categories']
        
        # Procesar imágenes del ground truth
        for img in gt_data.get('images', []):
            original_img_id = img['id']
            if original_img_id not in image_id_mapping:
                image_id_mapping[original_img_id] = next_image_id
                next_image_id += 1
                
            # Añadir imagen con ID global
            new_img = img.copy()
            new_img['id'] = image_id_mapping[original_img_id]
            global_gt['images'].append(new_img)
        
        # Procesar anotaciones del ground truth
        for ann in gt_data.get('annotations', []):
            original_img_id = ann['image_id']
            if original_img_id in image_id_mapping:
                # Añadir anotación con ID global
                new_ann = ann.copy()
                new_ann['id'] = next_annotation_id
                new_ann['image_id'] = image_id_mapping[original_img_id]
                global_gt['annotations'].append(new_ann)
                next_annotation_id += 1
        
        # Procesar predicciones
        for pred in video_predictions:
            original_img_id = pred['image_id']
            if original_img_id in image_id_mapping:
                # Añadir predicción con ID global
                new_pred = pred.copy()
                new_pred['image_id'] = image_id_mapping[original_img_id]
                global_predictions.append(new_pred)
    
    # Verificar que tenemos datos para evaluar
    if not global_predictions:
        raise ValueError("No se pudieron extraer predicciones con IDs de imagen válidos")
    
    if not global_gt['annotations']:
        raise ValueError("No se pudieron extraer anotaciones de ground truth con IDs de imagen válidos")
    
    # Estadísticas sobre los datos combinados
    num_predictions = len(global_predictions)
    num_images = len(global_gt['images'])
    num_annotations = len(global_gt['annotations'])
    
    print(f"Datos combinados: {num_predictions} predicciones, {num_images} imágenes, "
          f"{num_annotations} anotaciones ground truth")
    
    # Evaluar usando la API COCOeval
    coco_gt = COCO()
    coco_gt.dataset = global_gt
    coco_gt.createIndex()
    
    coco_dt = coco_gt.loadRes(global_predictions)
    
    # Inicializar evaluador
    coco_eval = COCOeval(coco_gt, coco_dt, iouType='segm')
    coco_eval.evaluate()
    coco_eval.accumulate()
    coco_eval.summarize()
    
    # Extraer métricas
    metrics = {
        'mAP': coco_eval.stats[0],  # mAP @ IoU=0.50:0.95
        'mAP_50': coco_eval.stats[1],  # mAP @ IoU=0.50
        'mAP_75': coco_eval.stats[2],  # mAP @ IoU=0.75
        'mAP_small': coco_eval.stats[3],  # mAP para objetos pequeños
        'mAP_medium': coco_eval.stats[4],  # mAP para objetos medianos
        'mAP_large': coco_eval.stats[5],  # mAP para objetos grandes
        'AR_max1': coco_eval.stats[6],  # AR dado 1 detección por imagen
        'AR_max10': coco_eval.stats[7],  # AR dado 10 detecciones por imagen
        'AR_max100': coco_eval.stats[8],  # AR dado 100 detecciones por imagen
        'AR_small': coco_eval.stats[9],  # AR para objetos pequeños
        'AR_medium': coco_eval.stats[10],  # AR para objetos medianos
        'AR_large': coco_eval.stats[11],  # AR para objetos grandes
        'num_videos': len(paired_files),  # Número de videos procesados
        'num_images': num_images,
        'num_predictions': num_predictions,
        'num_annotations': num_annotations
    }
    
    # Obtener categorías procesadas
    categories = []
    if global_gt['categories']:
        categories = [f"{cat['id']}: {cat['name']}" for cat in global_gt['categories']]
    
    # Escribir resultados en archivo TXT
    with open(output_file, 'w') as f:
        f.write("=" * 80 + "\n")
        f.write(f"RESULTADOS DE EVALUACIÓN DE INSTANCE SEGMENTATION\n")
        f.write(f"Fecha/Hora: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write("=" * 80 + "\n\n")
        
        f.write("RESUMEN DE DATOS\n")
        f.write("-" * 80 + "\n")
        f.write(f"Directorio de predicciones: {os.path.abspath(predictions_dir)}\n")
        f.write(f"Directorio de ground truth: {os.path.abspath(ground_truth_dir)}\n")
        
        # Añadir información sobre filtrado por video_ids si se proporcionó
        if video_ids:
            f.write(f"Filtrado por IDs de vídeo: {', '.join(video_ids)}\n")
            
        f.write(f"Número de videos procesados: {metrics['num_videos']}\n")
        f.write(f"Número total de imágenes: {metrics['num_images']}\n")
        f.write(f"Número total de predicciones: {metrics['num_predictions']}\n")
        f.write(f"Número total de anotaciones GT: {metrics['num_annotations']}\n\n")
        
        if categories:
            f.write("CATEGORÍAS\n")
            f.write("-" * 80 + "\n")
            for cat in categories:
                f.write(f"- {cat}\n")
            f.write("\n")
        
        f.write("ARCHIVOS PROCESADOS\n")
        f.write("-" * 80 + "\n")
        for file_pair in processed_files:
            f.write(f"{file_pair}\n")
        f.write("\n")
        
        f.write("MÉTRICAS GLOBALES\n")
        f.write("-" * 80 + "\n")
        f.write(f"mAP (IoU=0.50:0.95): {metrics['mAP']:.4f}\n")
        f.write(f"mAP (IoU=0.50): {metrics['mAP_50']:.4f}\n")
        f.write(f"mAP (IoU=0.75): {metrics['mAP_75']:.4f}\n\n")
        
        f.write("MÉTRICAS POR TAMAÑO DE OBJETO\n")
        f.write("-" * 80 + "\n")
        f.write(f"mAP para objetos pequeños: {metrics['mAP_small']:.4f}\n")
        f.write(f"mAP para objetos medianos: {metrics['mAP_medium']:.4f}\n")
        f.write(f"mAP para objetos grandes: {metrics['mAP_large']:.4f}\n\n")
        
        f.write("MÉTRICAS DE RECALL\n")
        f.write("-" * 80 + "\n")
        f.write(f"AR dado 1 detección por imagen: {metrics['AR_max1']:.4f}\n")
        f.write(f"AR dado 10 detecciones por imagen: {metrics['AR_max10']:.4f}\n")
        f.write(f"AR dado 100 detecciones por imagen: {metrics['AR_max100']:.4f}\n")
        f.write(f"AR para objetos pequeños: {metrics['AR_small']:.4f}\n")
        f.write(f"AR para objetos medianos: {metrics['AR_medium']:.4f}\n")
        f.write(f"AR para objetos grandes: {metrics['AR_large']:.4f}\n\n")
        
        f.write("=" * 80 + "\n")
        f.write("Fin del informe\n")
    
    print(f"\nInforme de resultados guardado en: {os.path.abspath(output_file)}")
    
    return metrics


In [15]:
gt_folder = '/Users/arnaubarrera/Desktop/MSc Computer Vision/C5. Visual Recognition/mcv-c5-group-3/huggingface/week2/Evaluation_off-the-shelf/ground_truth'
preds_folder = '/Users/arnaubarrera/Desktop/MSc Computer Vision/C5. Visual Recognition/mcv-c5-group-3/huggingface/week2/Evaluation_off-the-shelf/preds_off-the-shelf'

results_global = calculate_global_map_from_files(preds_folder, gt_folder, output_file="instance_segmentation_results.txt")

Encontrados 21 archivos de predicciones y 21 archivos de ground truth
Se emparejaron 21 archivos
Procesando: preds_coco_0000.json y gt_coco_0000.json
Procesando: preds_coco_0001.json y gt_coco_0001.json
Procesando: preds_coco_0002.json y gt_coco_0002.json
Procesando: preds_coco_0003.json y gt_coco_0003.json
Procesando: preds_coco_0004.json y gt_coco_0004.json
Procesando: preds_coco_0005.json y gt_coco_0005.json
Procesando: preds_coco_0006.json y gt_coco_0006.json
Procesando: preds_coco_0007.json y gt_coco_0007.json
Procesando: preds_coco_0008.json y gt_coco_0008.json
Procesando: preds_coco_0009.json y gt_coco_0009.json
Procesando: preds_coco_0010.json y gt_coco_0010.json
Procesando: preds_coco_0011.json y gt_coco_0011.json
Procesando: preds_coco_0012.json y gt_coco_0012.json
Procesando: preds_coco_0013.json y gt_coco_0013.json
Procesando: preds_coco_0014.json y gt_coco_0014.json
Procesando: preds_coco_0015.json y gt_coco_0015.json
Procesando: preds_coco_0016.json y gt_coco_0016.json
Pr

In [18]:
results_global = calculate_global_map_from_files(preds_folder, gt_folder, output_file="instance_segmentation_results.txt", video_ids=['0002'])

Encontrados 21 archivos de predicciones y 21 archivos de ground truth
Filtrando por 1 IDs de vídeo especificados
Se emparejaron 1 archivos
Procesando: preds_coco_0002.json y gt_coco_0002.json
Datos combinados: 873 predicciones, 218 imágenes, 1083 anotaciones ground truth
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *segm*
DONE (t=0.12s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.142
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.252
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.148
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.035
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.249
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.688
 Av